## Миграция данных о метро и запросы к БД: SQLite, JSON и Python

In [1]:
import sqlite3
from prettytable import from_db_cursor

In [2]:
db_name = 'mosmetro.db'

In [3]:
db = sqlite3.connect(db_name)
cur = db.cursor()

In [4]:
cur.execute('CREATE TABLE line (id integer PRIMARY KEY AUTOINCREMENT NOT NULL, name varchar(100) NOT NULL)')
db.commit()

In [5]:
cur.execute('CREATE TABLE area (id integer PRIMARY KEY AUTOINCREMENT NOT NULL, name varchar(100) NOT NULL)')
db.commit()

In [6]:
cur.execute('CREATE TABLE station (id integer PRIMARY KEY AUTOINCREMENT NOT NULL, name varchar(100) NOT NULL, id_line integer, id_area integer,'\
             'FOREIGN KEY (id_line) REFERENCES line (id), FOREIGN KEY (id_area) REFERENCES area (id))')
db.commit()

In [7]:
cur.execute("PRAGMA table_info('station')")
from_db_cursor(cur)

cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,1,None,1
1,name,varchar(100),1,None,0
2,id_line,INTEGER,0,None,0
3,id_area,INTEGER,0,None,0


In [8]:
cur.execute("PRAGMA table_info('line')")
from_db_cursor(cur)

cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,1,None,1
1,name,varchar(100),1,None,0


In [9]:
cur.execute("PRAGMA table_info('area')")
from_db_cursor(cur)

cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,1,None,1
1,name,varchar(100),1,None,0


In [10]:
import json

file_mosmetro_json = 'mosmetro.json'

with open(file_mosmetro_json) as f:
    d = json.load(f)

In [11]:
lines = []
for line in d:
    lines.append(line['Line'])
lines = list(set(lines))
lines

['Калининская линия',
 'Бутовская линия Лёгкого метро',
 'Кольцевая линия',
 'Замоскворецкая линия',
 'Некрасовская линия',
 'Арбатско-Покровская линия',
 'Люблинско-Дмитровская линия',
 'Таганско-Краснопресненская линия',
 'Калужско-Рижская линия',
 'Московская монорельсовая транспортная система',
 'Большая кольцевая линия',
 'Солнцевская линия',
 'Серпуховско-Тимирязевская линия',
 'Филёвская линия',
 'Московское центральное кольцо',
 'Сокольническая линия']

In [12]:
lines_data = [(line,) for line in lines]
lines_data

[('Калининская линия',),
 ('Бутовская линия Лёгкого метро',),
 ('Кольцевая линия',),
 ('Замоскворецкая линия',),
 ('Некрасовская линия',),
 ('Арбатско-Покровская линия',),
 ('Люблинско-Дмитровская линия',),
 ('Таганско-Краснопресненская линия',),
 ('Калужско-Рижская линия',),
 ('Московская монорельсовая транспортная система',),
 ('Большая кольцевая линия',),
 ('Солнцевская линия',),
 ('Серпуховско-Тимирязевская линия',),
 ('Филёвская линия',),
 ('Московское центральное кольцо',),
 ('Сокольническая линия',)]

In [13]:
cur.executemany("INSERT INTO line(name) VALUES(?)", lines_data)

In [14]:
cur.execute('SELECT * FROM line')
from_db_cursor(cur)

id,name
1,Калининская линия
2,Бутовская линия Лёгкого метро
3,Кольцевая линия
4,Замоскворецкая линия
5,Некрасовская линия
6,Арбатско-Покровская линия
7,Люблинско-Дмитровская линия
8,Таганско-Краснопресненская линия
9,Калужско-Рижская линия
10,Московская монорельсовая транспортная система


In [15]:
areas = []
for line in d:
    areas.append(line['AdmArea'])
areas = list(set(areas))
areas

['Южный административный округ',
 'Юго-Восточный административный округ',
 'Западный административный округ',
 'Центральный административный округ',
 'Юго-Западный административный округ',
 'Новомосковский административный округ',
 'Северный административный округ',
 'Восточный административный округ',
 'Северо-Восточный административный округ',
 'Северо-Западный административный округ']

In [16]:
areas_data = [(area,) for area in areas]
areas_data

[('Южный административный округ',),
 ('Юго-Восточный административный округ',),
 ('Западный административный округ',),
 ('Центральный административный округ',),
 ('Юго-Западный административный округ',),
 ('Новомосковский административный округ',),
 ('Северный административный округ',),
 ('Восточный административный округ',),
 ('Северо-Восточный административный округ',),
 ('Северо-Западный административный округ',)]

In [17]:
cur.executemany("INSERT INTO area(name) VALUES(?)", areas_data)

In [18]:
cur.execute('SELECT * FROM area')
from_db_cursor(cur)

id,name
1,Южный административный округ
2,Юго-Восточный административный округ
3,Западный административный округ
4,Центральный административный округ
5,Юго-Западный административный округ
6,Новомосковский административный округ
7,Северный административный округ
8,Восточный административный округ
9,Северо-Восточный административный округ
10,Северо-Западный административный округ


In [19]:
stations = []
for line in d:
    stations.append((line['Station'], int(''.join([f'{n+1}' for n, x in enumerate(lines) if line['Line'] == x])),\
                      int(''.join([f'{n+1}' for n, x in enumerate(areas) if line['AdmArea'] == x]))))
stations

[('Третьяковская', 1, 4),
 ('Медведково', 9, 9),
 ('Первомайская', 6, 8),
 ('Калужская', 9, 5),
 ('Каховская', 11, 5),
 ('Бульвар адмирала Ушакова', 2, 5),
 ('Павелецкая', 4, 4),
 ('Волжская', 7, 2),
 ('Шаболовская', 9, 1),
 ('Плющиха', 12, 4),
 ('Тимирязевская', 13, 7),
 ('ВДНХ', 9, 9),
 ('Тверская', 4, 4),
 ('Фонвизинская', 7, 9),
 ('Воронцовская', 11, 5),
 ('Дмитровская', 13, 9),
 ('Новые Черёмушки', 9, 5),
 ('Улица Милашенкова', 10, 9),
 ('Пушкинская', 8, 4),
 ('Маяковская', 4, 4),
 ('Нижегородская', 5, 2),
 ('Ботанический сад', 9, 9),
 ('Нижегородская', 11, 2),
 ('Белорусская', 4, 4),
 ('Красногвардейская', 4, 1),
 ('Комсомольская', 16, 4),
 ('Нагатинская', 13, 1),
 ('Алтуфьево', 13, 9),
 ('Новопеределкино', 12, 3),
 ('Автозаводская', 4, 1),
 ('Дорогомиловская', 12, 3),
 ('Текстильщики', 8, 2),
 ('Кожуховская', 7, 2),
 ('Марьино', 7, 2),
 ('Павелецкая', 3, 4),
 ('Преображенская площадь', 16, 8),
 ('Новослободская', 3, 4),
 ('Фили', 14, 3),
 ('Кунцевская', 11, 3),
 ('Домодедовская'

In [20]:
cur.executemany("INSERT INTO station(name, id_line, id_area) VALUES(?,?,?)", stations)

In [21]:
cur.execute('SELECT * FROM station')
from_db_cursor(cur)[:10]

id,name,id_line,id_area
1,Третьяковская,1,4
2,Медведково,9,9
3,Первомайская,6,8
4,Калужская,9,5
5,Каховская,11,5
6,Бульвар адмирала Ушакова,2,5
7,Павелецкая,4,4
8,Волжская,7,2
9,Шаболовская,9,1
10,Плющиха,12,4


In [22]:
# станции Арбатско-Покровской линии
cur.execute("SELECT station.name FROM station JOIN line ON station.id_line == line.id WHERE line.name == 'Арбатско-Покровская линия'")
from_db_cursor(cur)

name
Первомайская
Волоколамская
Парк Победы
Партизанская
Арбатская
Мякинино
Измайловская
Киевская
Крылатское
Площадь Революции


In [23]:
# станции Центрального административного округа
cur.execute("SELECT station.name FROM station JOIN area ON station.id_area == area.id WHERE area.name == 'Центральный административный округ'")
from_db_cursor(cur)[:10]

name
Третьяковская
Павелецкая
Плющиха
Тверская
Пушкинская
Маяковская
Белорусская
Комсомольская
Павелецкая
Новослободская


In [24]:
# станции Центрального административного округа и Арбатско-Покровской линии
cur.execute("SELECT station.name FROM station JOIN area ON station.id_area == area.id JOIN line ON \
            station.id_line == line.id WHERE area.name == 'Центральный административный округ' AND line.name == 'Арбатско-Покровская линия'")
from_db_cursor(cur)

name
Арбатская
Площадь Революции
Курская
Бауманская
Смоленская


In [25]:
# Количество станций на каждой ветке
cur.execute("SELECT line.name, COUNT(station.name) as 'count' FROM station JOIN line ON station.id_line == line.id GROUP BY line.name")
from_db_cursor(cur)

name,count
Арбатско-Покровская линия,22
Большая кольцевая линия,31
Бутовская линия Лёгкого метро,7
Замоскворецкая линия,24
Калининская линия,8
Калужско-Рижская линия,24
Кольцевая линия,13
Люблинско-Дмитровская линия,24
Московская монорельсовая транспортная система,6
Московское центральное кольцо,31


In [26]:
# Округ с максимальным количеством станций
cur.execute("SELECT area.name, COUNT(station.name) as 'count' FROM station JOIN area ON station.id_line == area.id \
            GROUP BY area.name ORDER BY count DESC LIMIT 1")
from_db_cursor(cur)

name,count
Центральный административный округ,24
